In [ ]:
import sys
from pathlib import Path
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
from context import Context
from utils.utils_chain import WrapperAddress


SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"

# SIMULATOR_URL = "https://proxy-shadowfork-four.elrond.ro"
# SIMULATOR_API = "https://proxy-shadowfork-four.elrond.ro"

GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL = f"{SIMULATOR_URL}/simulator/generate-blocks-until-epoch-reached"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)
DOCKER_URL = PROJECT_ROOT / "docker"

context = Context()
context.network_provider.proxy = proxy
context.network_provider.api = api
context.deployer_account.address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
context.deployer_account.sync_nonce(proxy)

wasm_path = "/home/multiversx/Documents/DEX/fees-collector/mx-sdk-py-exchange/wasm/fees-collector/fees-collector.wasm"
contract_code_hash = "ae5c0171d34f2bac384b6d38184f89abd7ef05a4d5d9b973db3c58015bdf587f"

## CHAIN SIM CONFIG - FEES COLLECTOR

In [ ]:
import json
import subprocess
from time import sleep
from typing import Any


def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    
def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", state)

# @pytest.fixture
def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        
def setup_chain_sim():
    # generate blocks to pass an epoch and the smart contract deploys to be enabled
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/5", {})

    load_and_apply_state(proxy, PROJECT_ROOT,
                         context.deployer_account.address.bech32(),
                         USERS)


def advance_blocks(number_of_blocks: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})

def advance_to_epoch(epoch: int):
    proxy.do_post(f"{GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL}/{epoch}", {})

def start_chain_sim_stack():
    # stop first in case one is already running
    p = subprocess.Popen(["docker", "compose", "down"], cwd = DOCKER_URL)
    p.wait()
    
    p = subprocess.Popen(["docker", "compose", "up", "-d"], cwd = DOCKER_URL)
    sleep(60)
    return p

def stop_chain_sim_stack(p):
    p.terminate()
    p = subprocess.Popen(["docker", "compose", "down"], cwd = DOCKER_URL)
    p.wait()
    _ = subprocess.run(["docker", "system", "prune", "-f"], cwd = DOCKER_URL)

In [ ]:
USERS = [
        "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw", 
        "erd1rv5twgkz5uatdvgk5ymzmgzmz38dxqh8agvlkt97mfun8hn4x4xqyptslm",
        "erd1njvcr0r89km6pexrxh0d6h36pkeuwr5j042e7l46l3mdlxvwhejsfz9w4n",
        "erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97" # DEX OWNER
]

BASE_TOKEN = "MEX-455c57"

In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97_0_chain_config_state.json", "r") as file:
    json_account = json.load(file)
proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", json_account)

In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_fees_collectors_0_chain_config_state.json", "r") as file:
    json_account = json.load(file)
proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", json_account)

In [ ]:
advance_epoch(7)

In [ ]:
start_chain_sim_stack()
setup_chain_sim()
initial_blocks = 10
advance_blocks(initial_blocks)

In [ ]:
advance_blocks(5)

In [ ]:
from contracts.fees_collector_contract import FeesCollectorContract
from contracts.pair_contract import PairContract
from utils.contract_retrievers import retrieve_pair_by_address
from utils.contract_data_fetchers import FeeCollectorContractDataFetcher
from contracts.router_contract import RouterContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]


pair_contract: PairContract
energy_contract: SimpleLockEnergyContract
router_contract: RouterContract

pair_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgq0lzzvt2faev4upyf586tg38s84d7zsaj2jpsglugga")    # operating pair
mex_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2") # egldmex contract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
router_contract = context.deploy_structure.get_deployed_contract_by_index(config.ROUTER_V2, 0)

print(fees_collector_contract.address)
print(pair_contract.address)
print(mex_contract.address)
print(router_contract.address)

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address

user_account = Account(context.deployer_account.address.to_bech32())
# "erd1rv5twgkz5uatdvgk5ymzmgzmz38dxqh8agvlkt97mfun8hn4x4xqyptslm"
# user_account.address = Address("erd1rv5twgkz5uatdvgk5ymzmgzmz38dxqh8agvlkt97mfun8hn4x4xqyptslm")
user_account.address = Address("erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw")
user_account.sync_nonce(context.network_provider.proxy)
user_account.signer = context.deployer_account.signer

In [ ]:
user_account.address = Address(USERS[0])
user_account.sync_nonce(context.network_provider.proxy)

fees_collector_contract.claim_rewards(user_account, proxy) #4 blocks
advance_blocks(4)

2025-04-24 09:22:27,588 - DEBUG - Account.sync_nonce() done: 4298
2025-04-24 09:22:27,589 - INFO - Claim rewards from fees collector
2025-04-24 09:22:27,590 - DEBUG - Calling claimRewards at erd1qqqqqqqqqqqqqpgqjsnxqprks7qxfwkcg2m2v9hxkrchgm9akp2segrswt
2025-04-24 09:22:27,591 - DEBUG - Args: []
2025-04-24 09:22:27,594 - DEBUG - Contract call arguments: []
2025-04-24 09:22:27,598 - INFO - No explorer known for http://localhost:8085. transaction raw path: http://localhost:8085/transaction/2f001cea114b8385f92f6b3d001412c755c4bb430f45556bf6bbbce5179c4d81


In [134]:
fees_collector_contract.claim_boosted_rewards(user_account, proxy)

2025-04-24 09:23:15,705 - INFO - Claim rewards from fees collector
2025-04-24 09:23:15,706 - DEBUG - Calling claimBoostedRewards at erd1qqqqqqqqqqqqqpgqjsnxqprks7qxfwkcg2m2v9hxkrchgm9akp2segrswt
2025-04-24 09:23:15,707 - DEBUG - Args: []
2025-04-24 09:23:15,711 - DEBUG - Contract call arguments: []
2025-04-24 09:23:15,715 - INFO - No explorer known for http://localhost:8085. transaction raw path: http://localhost:8085/transaction/1dae81fb1cd01661e799aafe1763a45922401f8e87b3da533046e2a2e2580549


'1dae81fb1cd01661e799aafe1763a45922401f8e87b3da533046e2a2e2580549'

In [ ]:
advance_blocks(1)

In [ ]:
from utils.utils_chain import WrapperAddress


context.deployer_account.address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.add_admin(context.deployer_account, proxy, [context.deployer_account.address])

In [ ]:
from utils.utils_chain import WrapperAddress


context.deployer_account.address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.redistribute_rewards(context.deployer_account, proxy)
advance_blocks(1)

SET ADMINS

In [ ]:
fees_collector_contract.add_admin(context.deployer_account, proxy, [context.deployer_account.address])

In [ ]:
advance_blocks(1)

In [123]:
week = fees_collector_contract.get_current_week(proxy)
user_energy = fees_collector_contract.get_user_energy_for_week(user_account.address.to_bech32(), proxy, week)
last_active_week = fees_collector_contract.get_last_active_week_for_user(user_account.address.to_bech32(), proxy)
print(week)
print(user_energy)
print(last_active_week)

126
{}
0


SET BURN PERCENTAGE

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 10000)
advance_blocks(1)

In [126]:
fees_collector_contract.deposit_swap_fees(context.deployer_account, proxy)
advance_blocks(1)

2025-04-24 09:21:29,792 - INFO - Deposit swap fees in fees collector
2025-04-24 09:21:29,794 - DEBUG - Calling depositSwapFees at erd1qqqqqqqqqqqqqpgqjsnxqprks7qxfwkcg2m2v9hxkrchgm9akp2segrswt
2025-04-24 09:21:29,794 - DEBUG - Args: []
2025-04-24 09:21:29,799 - DEBUG - Contract call arguments: []
2025-04-24 09:21:29,803 - INFO - No explorer known for http://localhost:8085. transaction raw path: http://localhost:8085/transaction/ab48d5f03e39b9db16bbf861fb84f196e08aefb50dcaeb99fa99f393ac2c5372


In [127]:
from utils.utils_chain import WrapperAddress, decode_merged_attributes
from utils import decoding_structures

data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
hex_result = data_fetcher.get_data("getTotalRewardsForWeek", [week])
print(hex_result)
decoded_results = decode_merged_attributes(hex_result, decoding_structures.TOTAL_REWARDS_FOR_WEEK)
print(decoded_results)

ValueError: invalid literal for int() with base 16: ''

In [128]:
token_identifier = "MEX-455c57"
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
hex_result = data_fetcher.get_data("getAccumulatedFees", [week, token_identifier])
print(hex_result)
# decoded_results = decode_merged_attributes(hex_result, decoding_structures.Ac)


0


In [129]:
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
hex_result = data_fetcher.get_data("getUserEnergyForWeek", [user_account.address, week])
print(hex_result)


In [ ]:
from utils.utils_chain import decode_merged_attributes
from utils import decoding_structures

data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
hex_result = data_fetcher.get_data("getTotalRewardsForWeek", [week])
print(hex_result)
decoded_results = decode_merged_attributes(hex_result, decoding_structures.ESDT_TOKEN_PAYMENT)
print(decoded_results)

GET CURRENT CLAIM PROGRESS

In [ ]:
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
hex_result = data_fetcher.get_data("getCurrentClaimProgress", [user_account.address])
decoded_results = decode_merged_attributes(hex_result, decoding_structures.USER_CLAIM_PROGRESS)
print(decoded_results)

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
advance_blocks(5)

In [ ]:
mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, pair_contract.address)

In [ ]:
# set where to swap and what to do with the fees
pair_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                    [
                                        mex_contract.address,
                                        mex_contract.firstToken,
                                        mex_contract.secondToken
                                    ])

In [ ]:
from contracts.pair_contract import AddLiquidityEvent

event = AddLiquidityEvent(pair_contract.firstToken, 127791780000000000000, 1, pair_contract.secondToken, 5000000000000000000, 1)
pair_contract.add_liquidity(context.network_provider, user_account, event)

In [ ]:
from utils.utils_chain import base64_to_hex


code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).code_hash
print(base64_to_hex(code_hash))

In [ ]:
advance_blocks(1)

In [ ]:
from contracts.pair_contract import SwapFixedInputEvent

user_account.address = Address("erd1njvcr0r89km6pexrxh0d6h36pkeuwr5j042e7l46l3mdlxvwhejsfz9w4n")
user_account.sync_nonce(proxy)
swap = SwapFixedInputEvent(pair_contract.firstToken, 1000000000000000000, pair_contract.secondToken, 1)

pair_contract.swap_fixed_input(context.network_provider, user_account, swap)
advance_blocks(1)

In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_system_account_state_WEGLD-bd4d79.json", "r") as file:
    json_account = json.load(file)
proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", json_account)

CONTRACT UPGRADE

In [ ]:
from utils.utils_chain import base64_to_hex
from utils.utils_chain import WrapperAddress

def fees_collector_upgrade():
    advance_blocks(1)
    context.deployer_account.address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
    context.deployer_account.sync_nonce(proxy)
    args = [ 
        energy_contract.address,
        router_contract.address,
        1000,
        []

    ]
    tx_hash = fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, wasm_path, 
                                         args, 
                                         no_init=True)

    advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).code_hash
    assert base64_to_hex(code_hash) == contract_code_hash
    
    return tx_hash

fees_collector_upgrade()

SWAP TO BASE TOKEN

In [ ]:
swapTokenToBaseToken = fees_collector_contract.swap_token(user_account, context.network_provider.proxy, pair_contract.secondToken, pair_contract)

In [ ]:
from multiversx_sdk import SmartContractTransactionsFactory, TransactionComputer
# from utils.utils_tx import get
from multiversx_sdk.abi import Abi


# logger = logger.get_logger(__name__)

network_config = context.network_provider.proxy.get_network_config()
abi = Abi.load(Path("/home/multiversx/Documents/DEX/fees-collector/mx-sdk-py-exchange/wasm/fees-collector/fees-collector.abi.json"))
sc_factory = SmartContractTransactionsFactory(network_config, abi)
context.deployer_account.address = Address("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
context.deployer_account.sync_nonce(proxy)
    
pair_address = Address(pair_contract.address)

def send_transaction():
    arguments: list[list[Any]] = []
    arguments.append((Address(pair_contract.address), "swapTokensFixedInput" , pair_contract.secondToken, 1))

    arguments.append((Address(mex_contract.address), "swapTokensFixedInput" , "MEX-455c57", 1))
    
    sc_args = [
           pair_contract.firstToken,
            arguments
        ]

    transaction = sc_factory.create_transaction_for_execute(
        sender=context.deployer_account.address,
        contract=Address(fees_collector_contract.address),
        function="swapTokenToBaseToken",
        gas_limit=50_000_000,
        arguments=sc_args
        )

    transaction.nonce = context.deployer_account.nonce

    # print(isinstance(ctor_data[0], bytes))

    transaction_computer = TransactionComputer()

    transaction.signature = context.deployer_account.signer.sign(
        transaction_computer.compute_bytes_for_signing(transaction)
    )

    print("Transaction:", transaction.__dict__)
    print("Transaction data:", transaction.data)

    result = proxy.send_transaction(transaction)
    print("Swap transaction hash:", result)

    return result

send_transaction()
advance_blocks(1)

In [ ]:
advance_blocks(1)

ADD REWARD TOKENS

In [ ]:
context.deployer_account.address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.add_reward_tokens(context.deployer_account, proxy, ["MEX-455c57", "XMEX-fda355"])

REMOVE REWARD TOKENS

In [ ]:
fees_collector_contract.remove_reward_tokens(context.deployer_account, proxy, ["USDC-c76f1f",
                                                                               "RIDE-7d18e9", 
                                                                               "CRU-a5f4aa", 
                                                                               "ZPAY-247875", 
                                                                               "ITHEUM-df6f26", 
                                                                               "BHAT-c1fde3", 
                                                                               "CRT-52decf", 
                                                                               "UTK-2f80e9",
                                                                               "QWT-46ac01",
                                                                               "ASH-a642d1",
                                                                               "WETH-b4ca29",
                                                                               "USDT-f8c08c",
                                                                               "HTM-f51d55",
                                                                               "WDAI-9eeb54",
                                                                               "TADA-5c032c",
                                                                               "XOXNO-c1293a",
                                                                               "A1X-0d446d",
                                                                               "USH-111e09",
                                                                               "FOXSY-5d5f3e"
                                                                               ])
advance_blocks(1)

In [ ]:
energy_factory: SimpleLockEnergyContract
energy_factory = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

context.deployer_account.sync_nonce(proxy)
tx_hash = energy_factory.set_burn_role_locked_token(context.deployer_account,
                                                                proxy,
                                                                [fees_collector_contract.address])

advance_blocks(1)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [mex_contract.address])
advance_blocks(1)

In [ ]:
from contracts.builtin_contracts import ESDTContract

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                               [BASE_TOKEN, fees_collector_contract.address, "ESDTRoleLocalMint", "ESDTRoleLocalBurn"])


In [ ]:
advance_blocks(1)